# Loading the Dataset

In [1]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [2]:
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

In [3]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Functional API implementation

## Multiple-input and Multiple-output (auxiliary-output) architecture

In [5]:
X_train.shape

(11610, 8)

In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
# For wide and deep path, choosing 5 features[0:4] through the wide path and choosing 6 features[2:7] through the deep path

input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_path')
hidden_1 = keras.layers.Dense(50, activation="relu")(input_B)
hidden_2 = keras.layers.Dense(40, activation="relu")(hidden_1)
concat = keras.layers.concatenate([input_A, hidden_2])
output = keras.layers.Dense(1, name="output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden_2)

model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [8]:
# 2 inputs and 2 outputs required
# since both inputs were defined with different features so creating new inputs accordingly

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

# new labels should be provided for all newly defined auxiliary outputs, but since in the model both the (old and new outputs) should try to predict same thing so we  directly passing 'y_train' twice instead of creating 'y_train_A' & 'y_train_B' explicitly.

In [9]:
# if a single loss= ... is passed then it will be applied to both the outputs, but main_output needs to have more weightage so loss_weight= ...

model.compile(loss=['mse', 'mse'], loss_weights=[0.9, 0.1], optimizer='sgd')

In [10]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=25, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/25
363/363 [==============================] - 3s 5ms/step - loss: 0.9785 - output_loss: 0.8788 - aux_output_loss: 1.8762 - val_loss: 0.6396 - val_output_loss: 0.5754 - val_aux_output_loss: 1.2172
Epoch 2/25
363/363 [==============================] - 1s 4ms/step - loss: 0.5703 - output_loss: 0.5137 - aux_output_loss: 1.0796 - val_loss: 0.5488 - val_output_loss: 0.4998 - val_aux_output_loss: 0.9900
Epoch 3/25
363/363 [==============================] - 1s 4ms/step - loss: 0.5267 - output_loss: 0.4821 - aux_output_loss: 0.9281 - val_loss: 0.5172 - val_output_loss: 0.4748 - val_aux_output_loss: 0.8990
Epoch 4/25
363/363 [==============================] - 2s 4ms/step - loss: 0.4749 - output_loss: 0.4370 - aux_output_loss: 0.8156 - val_loss: 0.4763 - val_output_loss: 0.4423 - val_aux_output_loss: 0.7828
Epoch 5/25
363/363 [==============================] - 1s 4ms/step - loss: 0.4544 - output_loss: 0.4232 - aux_output_loss: 0.7353 - val_loss: 0.4628 - val_output_loss: 0.4338 - val_aux_

In [11]:
# we get total loss and individual losses as well

total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

162/162 [==============================] - 0s 3ms/step - loss: 0.3465 - output_loss: 0.3330 - aux_output_loss: 0.4686


In [12]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

1/1 [==============================] - 0s 151ms/step


## Saving and Restoring the Model

In [13]:
model.save("my_keras_model.h5")

# this can save all the parameters, hyperparameters and optimizers for 'Sequential' and 'Functional' APIs. (not for Subclassing APIs)

In [14]:
model = keras.models.load_model("my_keras_model.h5")

## Callbacks

### ModelCheckpoint Callback

In [15]:
modelcheckpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# if save_best_only = False (default), then we will have a checkpoint after each epoch (by default) hence provide a formate to save each of those files preferably in the order of {epoch_no}_{val_loss}

In [16]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=25, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]), callbacks=[modelcheckpoint_cb])

Epoch 1/25
360/363 [============================>.] - ETA: 0s - loss: 0.3458 - output_loss: 0.3332 - aux_output_loss: 0.4594
Epoch 1: val_loss improved from inf to 0.36718, saving model to my_keras_model.h5
363/363 [==============================] - 2s 5ms/step - loss: 0.3459 - output_loss: 0.3333 - aux_output_loss: 0.4595 - val_loss: 0.3672 - val_output_loss: 0.3553 - val_aux_output_loss: 0.4745
Epoch 2/25
356/363 [============================>.] - ETA: 0s - loss: 0.3434 - output_loss: 0.3310 - aux_output_loss: 0.4549
Epoch 2: val_loss improved from 0.36718 to 0.36053, saving model to my_keras_model.h5
363/363 [==============================] - 2s 6ms/step - loss: 0.3433 - output_loss: 0.3309 - aux_output_loss: 0.4553 - val_loss: 0.3605 - val_output_loss: 0.3493 - val_aux_output_loss: 0.4614
Epoch 3/25
351/363 [============================>.] - ETA: 0s - loss: 0.3428 - output_loss: 0.3306 - aux_output_loss: 0.4519
Epoch 3: val_loss did not improve from 0.36053
363/363 [===============

In [17]:
model = keras.models.load_model("my_keras_model.h5")

# rolling back to the best model with 'minimum' 'val_loss' value saved and overwritten as 'my_keras_model'

In [18]:
history.history

{'loss': [0.3459203839302063,
  0.343308687210083,
  0.34223008155822754,
  0.3387058675289154,
  0.3387146592140198,
  0.3378479778766632,
  0.3521268665790558,
  0.3489410877227783,
  0.33826178312301636,
  0.33370441198349,
  0.33250391483306885,
  0.33119872212409973,
  0.3292163908481598,
  0.3327263593673706,
  0.331166535615921,
  0.3285514712333679,
  0.3271704614162445,
  0.3261755406856537,
  0.32267045974731445,
  0.3236371576786041,
  0.3240714371204376,
  0.32286536693573,
  0.3243517279624939,
  0.33035773038864136,
  0.3203165829181671],
 'output_loss': [0.33330368995666504,
  0.3308705687522888,
  0.3300817310810089,
  0.3267561197280884,
  0.3272000551223755,
  0.3266880512237549,
  0.341232568025589,
  0.33809301257133484,
  0.32783061265945435,
  0.32338839769363403,
  0.3223128914833069,
  0.3212641775608063,
  0.3193570375442505,
  0.3234739899635315,
  0.3220181167125702,
  0.3192876875400543,
  0.3180919885635376,
  0.3172188997268677,
  0.3136448860168457,
  0.3

### EarlyStopping Callback

In [19]:
## EarlyStopping callback interrupts training if no progress on val_set for a given no. of epochs (patience number) and roll backs to the best model.

In [20]:
earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True, verbose=1)
# 'restore_best_weights = True' to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used.

In [22]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs = 100, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]), callbacks=[modelcheckpoint_cb, earlystopping_cb])

# no. of epochs can be set very high since training will stop automatically if no more progress.
# no need to restore best model saved as EarlyStopping_cb will keep track of the best weights and restore them at the end of training.

Epoch 1/100
347/363 [===========================>..] - ETA: 0s - loss: 0.3013 - output_loss: 0.2949 - aux_output_loss: 0.3594
Epoch 1: val_loss did not improve from 0.31522
363/363 [==============================] - 2s 5ms/step - loss: 0.3025 - output_loss: 0.2960 - aux_output_loss: 0.3611 - val_loss: 0.3212 - val_output_loss: 0.3153 - val_aux_output_loss: 0.3741
Epoch 2/100
347/363 [===========================>..] - ETA: 0s - loss: 0.3035 - output_loss: 0.2968 - aux_output_loss: 0.3635
Epoch 2: val_loss did not improve from 0.31522
363/363 [==============================] - 1s 4ms/step - loss: 0.3052 - output_loss: 0.2987 - aux_output_loss: 0.3643 - val_loss: 0.3275 - val_output_loss: 0.3217 - val_aux_output_loss: 0.3793
Epoch 3/100
354/363 [============================>.] - ETA: 0s - loss: 0.3014 - output_loss: 0.2948 - aux_output_loss: 0.3612
Epoch 3: val_loss did not improve from 0.31522
363/363 [==============================] - 2s 5ms/step - loss: 0.3016 - output_loss: 0.2950 - a